In [1]:
'''
this code is for adding statistical columns to the our dataframes
'''
import os
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
#from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_absolute_percentage_error
from datetime import datetime
import threading



In [2]:
def dropColumns(df):
    '''
    used graner causaliry test in order to choose base features
    data = dataset[['avg_cpu_load_5', 'cpu_util']].pct_change()
    data.replace([np.inf, -np.inf], np.nan, inplace=True)
    data=data.dropna()
    gc_res = grangercausalitytests(data,12)
    '''
    df.drop(['instance_id','disk_util','avg_cpi','avg_mpki','max_cpi','max_mpki'] , axis=1, inplace=True)
    return df


In [3]:
def statisticalColumns(df):
    '''
    adds statistical columns to data
    ma6 = movinga average with 6 past windows
    ma12 = movinga average with 12 past windows
    ema = exponential moving average
    '''
    df['ma6'] = df['cpu_util'].rolling(window = 6).mean()
    df['ma12'] = df['cpu_util'].rolling(window = 12).mean()
    df['ema'] = df['cpu_util'].ewm(com=0.7).mean()
    df.interpolate(limit_direction="both",inplace=True)
    return df

In [4]:
def getArimaHyperparamters(timeseries):
    '''
    returns differencing level for arima column
    '''
    values = timeseries.values
    iterration = 0
    p_values = []
    adfuller_result = adfuller(values)
    p_values.append(adfuller_result[1])
    while adfuller_result[1] > 0.5 and iterration <=5 :
        timeseries = timeseries.diff().fillna(method='ffill').fillna(method='bfill')
        values= timeseries.values
        adfuller_result = adfuller(values)
        p_values.append(adfuller_result[1])
        iterration += 1
    min_value = min(p_values)
    arima_d = p_values.index(min_value)   
    if arima_d == 0:
        arima_d +=1
    return arima_d 

In [5]:
def arimaModel(timeseries_data):
    ''' 
    creates arima predictions for the 48 last lines of the dataframe
    '''
    arima_d = getArimaHyperparamters(timeseries_data)
    timeseries_data = timeseries_data.values
    size = int(len(timeseries_data) * 0.66)
    train, test = timeseries_data[0:size], timeseries_data[size:len(timeseries_data)]
    history = [x for x in train]
    predictions = list()
    for t in range(len(test)):
        model = ARIMA(history, order=(12,arima_d,0))
        model.initialize_approximate_diffuse()
        model_fit = model.fit()
        output = model_fit.forecast()
        yhat = output[0]
        predictions.append(yhat)
        obs = test[t]
        history.append(obs)
    arima_prediction = pd.DataFrame(predictions)
    error = mean_absolute_percentage_error(test, predictions)*100
    return arima_prediction , error

In [6]:
def arimaColumn(df):
    '''
    adds arima predictions as column to our dataset
    '''
    cpu_util_series = df['cpu_util']
    arima_prediction, error = arimaModel(cpu_util_series)
    df.loc[df.index[-len(arima_prediction):],'ARIMA'] = arima_prediction.values
    df.loc[df.index[:-len(arima_prediction)], 'ARIMA'] = cpu_util_series[:-len(arima_prediction)]
    return df

In [7]:
def alterDataset(df):
    altereddf = dropColumns(df)
    altereddf = statisticalColumns(altereddf)
    altereddf = arimaColumn(altereddf)
    return altereddf

In [16]:
'''
variables
'''
error_list =[]
mySeries = []
namesOfMySeries = []
alteredSeries=[]

In [17]:
directory = 'D:\\master\\05\\project\\data\\2017\\engineered_data\\'
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        alteredSeries.append(filename[:-4])

In [18]:
len(alteredSeries)

10352

In [19]:
directory = 'D:\\master\\05\\project\\data\\2017\\seperate_datasets\\'

for filename in os.listdir(directory):
    
    if filename.endswith(".csv") :
        
        if filename[:-4] not in alteredSeries:
            df = pd.read_csv(directory+filename, header=0, index_col=0)
            mySeries.append(df)
            namesOfMySeries.append(filename[:-4])
        

In [20]:
len(namesOfMySeries)

0

In [13]:
len(mySeries)

565

In [14]:
def creatFiles(mySeries,namesOfMySeries):
    for i in range(len(mySeries)):
        print(' ***processing file  ::'+namesOfMySeries[i])
        print(datetime.now())
        temp = alterDataset(mySeries[i])
        temp.to_csv('D:\\master\\05\\project\\data\\2017\\engineered_data\\'+namesOfMySeries[i]+'.csv')

In [15]:
creatFiles(mySeries= mySeries, namesOfMySeries= namesOfMySeries)

 ***processing file  ::container_282
2023-02-16 00:52:33.035087


C:\Users\Lenovo\AppData\Roaming\Python\Python38\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


 ***processing file  ::container_2820
2023-02-16 00:52:58.518520
 ***processing file  ::container_2821
2023-02-16 00:53:22.992039


C:\Users\Lenovo\AppData\Roaming\Python\Python38\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\Lenovo\AppData\Roaming\Python\Python38\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\Lenovo\AppData\Roaming\Python\Python38\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\Lenovo\AppData\Roaming\Python\Python38\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\Lenovo\AppData\

 ***processing file  ::container_2822
2023-02-16 00:53:58.863861


...

C:\Users\Lenovo\AppData\Roaming\Python\Python38\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\Lenovo\AppData\Roaming\Python\Python38\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


 ***processing file  ::container_8377
2023-02-16 04:00:34.505423
 ***processing file  ::container_8378
2023-02-16 04:00:52.286343


C:\Users\Lenovo\AppData\Roaming\Python\Python38\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\Lenovo\AppData\Roaming\Python\Python38\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\Lenovo\AppData\Roaming\Python\Python38\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\Lenovo\AppData\Roaming\Python\Python38\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


 ***processing file  ::container_8379
2023-02-16 04:01:15.202728


C:\Users\Lenovo\AppData\Roaming\Python\Python38\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
